In [3]:
import numpy as np
import pandas as pd


In [40]:
df = pd.read_csv('/content/diabetes.csv')

In [41]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [42]:
df.corr()['Outcome'].sort_values(ascending=False)

,Outcome
Outcome,1.000000
Glucose,0.466581
BMI,0.292695
Age,0.238356
Pregnancies,0.221898
DiabetesPedigreeFunction,0.173844
Insulin,0.130548
SkinThickness,0.074752
BloodPressure,0.065068


In [43]:
df.shape

(768, 9)

In [44]:
X=df.iloc[:,:-1].values
y=df.iloc[:,-1].values

In [45]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [46]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [47]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [48]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [49]:
model = Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [50]:
model.fit(X_train_scaled,y_train,batch_size=32, epochs=100,validation_data=(X_test_scaled,y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.4622 - loss: 0.7240 - val_accuracy: 0.6883 - val_loss: 0.6554
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6300 - loss: 0.6550 - val_accuracy: 0.7468 - val_loss: 0.6105
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7273 - loss: 0.6083 - val_accuracy: 0.7273 - val_loss: 0.5827
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7397 - loss: 0.5799 - val_accuracy: 0.7532 - val_loss: 0.5584
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7469 - loss: 0.5513 - val_accuracy: 0.7662 - val_loss: 0.5435
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7737 - loss: 0.5320 - val_accuracy: 0.7662 - val_loss: 0.5330
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7265 - loss: 0.5464 - val_accuracy: 0.7857 - val_loss: 0.5234
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7865 - loss: 0.4929 - val_accuracy: 0.7922 - 

In [51]:
pip install -U keras-tuner

In [52]:
import kerastuner as kt

In [53]:
def build_model (hp):

  model=Sequential()
  model.add(Dense(32,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  optimizer=hp.Choice('optimizer',values = ['adam','sgd','rmsprop','adadelta'])

  model.compile(optimizer=optimizer, loss='binary_crossentropy',metrics=['accuracy'])

  return model



In [54]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [55]:
tuner.search(X_train_scaled,y_train,epochs=5,validation_data=(X_test_scaled,y_test))

In [34]:
best_hps = tuner.get_best_hyperparameters(num_trials=5)[0]
print("Best Hyperparameters:")
print(f"Optimizer: {best_hps.get('optimizer')}")


Best Hyperparameters:
Optimizer: adam


In [32]:
best_hps